In [1]:
# BAGLS sequence organizer using metadata to derandomize the shuffled frames.

import os
import json
import shutil
from glob import glob
from PIL import Image

def organize_bagls_triplets_by_sequence(src_dir, dst_root):
    """
    Organizes BAGLS triplets (x.png, x_seg.png, x.meta) into folders named like:
        "HxW - VideoId"
    where:
        - HxW is resolution from .meta (Video resolution (px, HxW))
        - VideoId is 'Video Id' from .meta
    
    Args:
        src_dir  : folder containing all mixed triplets
        dst_root : output root directory for structured folders
    """
    os.makedirs(dst_root, exist_ok=True)

    meta_paths = sorted(glob(os.path.join(src_dir, "*.meta")))
    moved = 0
    missing = 0

    for meta_path in meta_paths:
        stem = os.path.splitext(os.path.basename(meta_path))[0]
        img_path = os.path.join(src_dir, f"{stem}.png")
        seg_path = os.path.join(src_dir, f"{stem}_seg.png")

        # skip if any file is missing
        if not (os.path.exists(img_path) and os.path.exists(seg_path)):
            print(f"⚠️ Missing for {stem}, skipping.")
            missing += 1
            continue

        # read meta info
        try:
            with open(meta_path, "r", encoding="utf-8") as f:
                meta = json.load(f)
            vid = meta.get("Video Id", "unknown")
            res = meta.get("Video resolution (px, HxW)", None)
            if isinstance(res, list) and len(res) == 2:
                H, W = res
                res_str = f"{H}x{W}"
            else:
                # fallback: read from actual image
                with Image.open(img_path) as im:
                    W, H = im.size
                res_str = f"{H}x{W}"
        except Exception as e:
            print(f"❌ Error reading {meta_path}: {e}")
            continue

        folder_name = f"{res_str} - {vid}"
        dst_folder = os.path.join(dst_root, folder_name)
        os.makedirs(dst_folder, exist_ok=True)

        # move or copy files
        for p in [img_path, seg_path, meta_path]:
            dst_path = os.path.join(dst_folder, os.path.basename(p))
            shutil.move(p, dst_path)
            moved += 1

    print(f"✅ Organized {len(meta_paths)} triplets. Files moved: {moved}. Missing: {missing}.")

In [2]:

organize_bagls_triplets_by_sequence(
    src_dir=r"D:\bagls-data\BAGLS\test",
    dst_root=r"D:\bagls-data\BAGLS\test_organized"
)


✅ Organized 3500 triplets. Files moved: 10500. Missing: 0.


In [3]:
# The function crop_bagls_256 reshapes BAGLS frames of various resolutions / framings / channels to one 256x256 RGB Fehling-like standard.

import os
from glob import glob
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

# ------------------------- helpers -------------------------

def _is_grayscale_image(pil_img):
    """Heuristic: 'L' mode or nearly-equal RGB channels on a downsampled sample."""
    if pil_img.mode == "L":
        return True
    if pil_img.mode != "RGB":
            # treat non-RGB, non-L as color (we'll convert on save if requested)
        return False
    arr = np.array(pil_img.resize((32, 32)))
    return np.allclose(arr[...,0], arr[...,1]) and np.allclose(arr[...,1], arr[...,2])

def _load_pair(img_path):
    seg_path = img_path.replace(".png", "_seg.png")
    # image
    img = Image.open(img_path)
    if img.mode == "L":
        img_np = np.array(img)  # (H,W)
    else:
        img = img.convert("RGB")
        img_np = np.array(img)  # (H,W,3)
    # mask -> binary
    seg = Image.open(seg_path).convert("L")
    seg_np = (np.array(seg) > 0).astype(np.uint8)
    return img_np, seg_np

def _save_pair(out_img_path, out_seg_path, img_np, seg_np, to_rgb=False):
    # ensure dtype
    if img_np.dtype != np.uint8:
        img_np = img_np.astype(np.uint8)
    if seg_np.dtype != np.uint8:
        seg_np = seg_np.astype(np.uint8)

    # image save (stack grayscale to rgb if requested)
    if img_np.ndim == 2:
        if to_rgb:
            img_np = np.stack([img_np]*3, axis=-1)
        Image.fromarray(img_np).save(out_img_path)
    else:
        Image.fromarray(img_np).save(out_img_path)

    # mask save (255 for foreground, 0 for background)
    Image.fromarray((seg_np * 255).astype(np.uint8)).save(out_seg_path)

def _nonblack_content_bbox(img_np, thr=10):
    """BBox of non-black content. Works for 1 or 3 channels. Returns (y0,x0,y1,x1)."""
    if img_np.ndim == 3:
        gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    else:
        gray = img_np
    _, mask = cv2.threshold(gray, thr, 255, cv2.THRESH_BINARY)
    ys, xs = np.where(mask > 0)
    if len(xs) == 0:
        H, W = gray.shape[:2]
        return (0,0,H-1,W-1)
    return (int(ys.min()), int(xs.min()), int(ys.max()), int(xs.max()))

def _glottis_centroid(seg_np):
    """Return (cy, cx) or None."""
    ys, xs = np.where(seg_np > 0)
    if len(xs) == 0:
        return None
    return (float(np.mean(ys)), float(np.mean(xs)))

def _avg_glottis_center_in_folder(stems, folder):
    cys, cxs = [], []
    for s in stems:
        sp = os.path.join(folder, f"{s}_seg.png")
        if not os.path.exists(sp):
            continue
        seg = (np.array(Image.open(sp).convert("L")) > 0).astype(np.uint8)
        c = _glottis_centroid(seg)
        if c is not None:
            cys.append(c[0]); cxs.append(c[1])
    if cys:
        return (float(np.mean(cys)), float(np.mean(cxs)))
    return None

def _square_crop_centered(H, W, center, size=256):
    """Return y0,y1,x0,x1 for a size×size crop centered near 'center', clamped to image."""
    cy, cx = center
    half = size // 2
    y0 = max(0, min(H - size, int(round(cy)) - half))
    x0 = max(0, min(W - size, int(round(cx)) - half))
    y1 = y0 + size
    x1 = x0 + size
    return y0, y1, x0, x1

def _copy_if_needed(arr):
    return arr.copy() if not arr.flags.writeable else arr

# ------------------------- main -------------------------

def crop_bagls_256(root_organized_dir, transform_to_rgb=True):
    """
    For each video folder 'HxW - VideoId' under root_organized_dir:
      - Create/overwrite 'croped_256' subfolder.
      - Process each (img, seg) pair to produce 256x256 outputs, following rules:

    Rules implemented:
      1) 256xN (N<256): pad left/right with black to reach 256x256.
      2) Nx256 (N>256) or NxM where both >256: trim black borders (circular FOV),
         then crop a 256x256 square biased toward the glottis. If no glottis in frame,
         use per-video avg glottis center; else image center.
      3) No-glottis frames: use per-video avg glottis center.
      4) No geometric distortion (no non-uniform scaling). Only crop/pad (and 512->256 scale).
      5) 512x512: uniform downscale to 256x256 (INTER_AREA for image, NEAREST for mask).
      6) Already 256x256: copy as-is into 'croped_256'.
      7) Do NOT copy metas.
      8) Optional: per-video grayscale->RGB stacking (transform_to_rgb).
    """
    video_dirs = [d for d in sorted(glob(os.path.join(root_organized_dir, "*"))) if os.path.isdir(d)]
    for vdir in tqdm(video_dirs, desc="Videos"):
        # collect frame stems in this video folder
        stems = []
        for p in sorted(glob(os.path.join(vdir, "*.png"))):
            base = os.path.basename(p)
            if base.endswith("_seg.png"):
                continue
            stems.append(os.path.splitext(base)[0])
        if not stems:
            continue

        # per-video grayscale decision (first frame only)
        to_rgb = False
        if transform_to_rgb:
            first_img_path = os.path.join(vdir, f"{stems[0]}.png")
            if os.path.exists(first_img_path):
                to_rgb = _is_grayscale_image(Image.open(first_img_path))

        # per-video average glottis center (in ORIGINAL coords)
        avg_center = _avg_glottis_center_in_folder(stems, vdir)

        out_dir = os.path.join(vdir, "croped_256")
        os.makedirs(out_dir, exist_ok=True)

        for s in stems:
            img_path = os.path.join(vdir, f"{s}.png")
            seg_path = os.path.join(vdir, f"{s}_seg.png")
            if not os.path.exists(seg_path):
                continue

            out_img = os.path.join(out_dir, f"{s}.png")
            out_seg = os.path.join(out_dir, f"{s}_seg.png")

            img_np, seg_np = _load_pair(img_path)
            H, W = img_np.shape[:2]

            # Rule 5: exact 512x512 -> uniform downscale to 256x256
            if H == 512 and W == 512:
                img_256 = cv2.resize(img_np, (256, 256), interpolation=cv2.INTER_AREA)
                seg_256 = cv2.resize(seg_np, (256, 256), interpolation=cv2.INTER_NEAREST)
                _save_pair(out_img, out_seg, img_256, seg_256, to_rgb)
                continue

            # Rule 6: already 256x256 -> copy
            if H == 256 and W == 256:
                _save_pair(out_img, out_seg, img_np, seg_np, to_rgb)
                continue

            # Trim black borders first (handles circular FOV etc.)
            y0b, x0b, y1b, x1b = _nonblack_content_bbox(img_np)
            img_trim = _copy_if_needed(img_np)[y0b:y1b+1, x0b:x1b+1]
            seg_trim = seg_np[y0b:y1b+1, x0b:x1b+1]
            Ht, Wt = img_trim.shape[:2]

            # Preferred center = glottis centroid (in trimmed coords)
            c = _glottis_centroid(seg_trim)
            if c is None:
                # Use per-video average (projected to trimmed coordinates) if present
                if avg_center is not None:
                    cy_avg, cx_avg = avg_center
                    c = (max(0, min(Ht - 1, cy_avg - y0b)),
                         max(0, min(Wt - 1, cx_avg - x0b)))
                else:
                    c = (Ht / 2.0, Wt / 2.0)

            # Rule 1: 256xN (N<256) after trim -> pad left/right to 256
            if Ht == 256 and Wt < 256:
                pad_total = 256 - Wt
                # bias center toward glottis: place it near 128 horizontally
                left = int(min(max(0, int(round(128 - c[1]))), pad_total))
                right = pad_total - left
                if img_trim.ndim == 3:
                    img_256 = cv2.copyMakeBorder(img_trim, 0, 0, left, right,
                                                 cv2.BORDER_CONSTANT, value=(0, 0, 0))
                else:
                    img_256 = cv2.copyMakeBorder(img_trim, 0, 0, left, right,
                                                 cv2.BORDER_CONSTANT, value=0)
                seg_256 = cv2.copyMakeBorder(seg_trim, 0, 0, left, right,
                                             cv2.BORDER_CONSTANT, value=0)
                _save_pair(out_img, out_seg, img_256, seg_256, to_rgb)
                continue

            # Case: width is exactly 256 and height > 256 -> crop vertically around c
            if Wt == 256 and Ht > 256:
                y0, y1, x0, x1 = _square_crop_centered(Ht, Wt, c, 256)
                img_256 = img_trim[y0:y1, x0:x1]
                seg_256 = seg_trim[y0:y1, x0:x1]
                _save_pair(out_img, out_seg, img_256, seg_256, to_rgb)
                continue

            # Case: both >= 256 -> crop a 256x256 square centered near c
            if Ht >= 256 and Wt >= 256:
                y0, y1, x0, x1 = _square_crop_centered(Ht, Wt, c, 256)
                img_256 = img_trim[y0:y1, x0:x1]
                seg_256 = seg_trim[y0:y1, x0:x1]
                _save_pair(out_img, out_seg, img_256, seg_256, to_rgb)
                continue

            # --- Mixed small/large cases WITHOUT distortion (patched clamps) ---
            img_work = img_trim
            seg_work = seg_trim

            # Ensure height = 256 by crop or pad
            if Ht >= 256:
                # crop vertically around c[0]
                y0 = max(0, min(Ht - 256, int(round(c[0])) - 128))
                y1 = y0 + 256
                img_work = img_work[y0:y1, :]
                seg_work = seg_work[y0:y1, :]
                Ht = 256
                # center changes but not needed further for height
            else:
                # pad height to 256 (black), clamp top within budget
                pad_total = 256 - Ht
                top = int(min(max(0, int(round(128 - c[0]))), pad_total))
                bottom = pad_total - top
                if img_work.ndim == 3:
                    img_work = cv2.copyMakeBorder(img_work, top, bottom, 0, 0,
                                                  cv2.BORDER_CONSTANT, value=(0,0,0))
                else:
                    img_work = cv2.copyMakeBorder(img_work, top, bottom, 0, 0,
                                                  cv2.BORDER_CONSTANT, value=0)
                seg_work = cv2.copyMakeBorder(seg_work, top, bottom, 0, 0,
                                              cv2.BORDER_CONSTANT, value=0)
                Ht = 256
                c = (c[0] + top, c[1])  # update center after padding

            # Ensure width = 256 by crop or pad
            if Wt >= 256:
                # crop horizontally around c[1]
                x0 = max(0, min(Wt - 256, int(round(c[1])) - 128))
                x1 = x0 + 256
                img_work = img_work[:, x0:x1]
                seg_work = seg_work[:, x0:x1]
            else:
                pad_total = 256 - Wt
                left = int(min(max(0, int(round(128 - c[1]))), pad_total))
                right = pad_total - left
                if img_work.ndim == 3:
                    img_work = cv2.copyMakeBorder(img_work, 0, 0, left, right,
                                                  cv2.BORDER_CONSTANT, value=(0,0,0))
                else:
                    img_work = cv2.copyMakeBorder(img_work, 0, 0, left, right,
                                                  cv2.BORDER_CONSTANT, value=0)
                seg_work = cv2.copyMakeBorder(seg_work, 0, 0, left, right,
                                              cv2.BORDER_CONSTANT, value=0)

            img_256, seg_256 = img_work, seg_work
            _save_pair(out_img, out_seg, img_256, seg_256, to_rgb)

    print("✅ Done: all videos have croped_256 outputs.")

In [4]:
crop_bagls_256(r"D:\bagls-data\BAGLS\test_organized")

Videos: 100%|██████████████████████████████████████████████████████████████████████████| 70/70 [01:31<00:00,  1.31s/it]

✅ Done: all videos have croped_256 outputs.


In [12]:
import os
import shutil

def wipe_croped_256(root_organized_dir):
    """
    Recursively removes all 'croped_256' subfolders inside the given root directory.
    """
    removed = 0
    for dirpath, dirnames, filenames in os.walk(root_organized_dir):
        for d in dirnames:
            if d.lower() == "croped_256":   # case-insensitive safety
                full_path = os.path.join(dirpath, d)
                print(f"🗑 Removing: {full_path}")
                shutil.rmtree(full_path)
                removed += 1
    print(f"✅ Done. Removed {removed} 'croped_256' folders.")

# Example usage:
wipe_croped_256(r"D:\bagls-data\BAGLS\training\organized")

🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 117\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 124\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 334\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 399\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 41\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 43\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 445\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 477\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 478\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 500\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 632\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x120 - 86\croped_256
🗑 Removing: D:\bagls-data\BAGLS\training\organized\256x256 - 113\croped_256
🗑 Removing: D:\

In [13]:
crop_bagls_256(r"D:\bagls-data\BAGLS\training\organized")

Videos: 100%|████████████████████████████████████████████████████████████████████████| 570/570 [23:12<00:00,  2.44s/it]

✅ Done: all videos have croped_256 outputs.


In [5]:
# An algorithm that is ordering shuffled frames within each sequence based on the glottis size / location

import os
import csv
import uuid
from glob import glob
from typing import List, Tuple, Dict, Optional
import numpy as np

try:
    import cv2
except ImportError:
    raise ImportError("This script needs OpenCV (cv2). Install with: pip install opencv-python")

# =========================
# Utilities
# =========================

def _read_png_gray(path: str) -> np.ndarray:
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Could not read image: {path}")
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def _extract_glottis_binary(mask_img: np.ndarray) -> np.ndarray:
    # treat any non-zero as glottis
    return (mask_img > 0).astype(np.uint8)

def _maybe_dilate(binary_mask: np.ndarray, dilate_px: int) -> np.ndarray:
    if dilate_px and dilate_px > 0:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*dilate_px+1, 2*dilate_px+1))
        return cv2.dilate(binary_mask, k, iterations=1)
    return binary_mask

def dice_coef(m1: np.ndarray, m2: np.ndarray, eps: float = 1e-6) -> float:
    inter = np.sum(m1 & m2)
    s1 = np.sum(m1)
    s2 = np.sum(m2)
    return (2.0 * inter + eps) / (s1 + s2 + eps)

def centroid(mask_bin: np.ndarray) -> tuple[float, float]:
    ys, xs = np.nonzero(mask_bin)
    if ys.size == 0:
        return (-1.0, -1.0)
    return (float(ys.mean()), float(xs.mean()))

# =========================
# Post-ordering area smoother (new)
# =========================

def _area_series(index_seq: List[int], areas: Dict[int, int]) -> np.ndarray:
    return np.array([areas[i] for i in index_seq], dtype=np.float64)

def _robust_jump_threshold(deltas: np.ndarray) -> float:
    """
    Robust threshold for 'sudden jump' detection on normalized area deltas.
    thr = max(median + 3*MAD, 0.08), then clipped to 0.6.
    """
    if deltas.size == 0:
        return 1.0
    med = np.median(deltas)
    mad = np.median(np.abs(deltas - med))
    thr = max(med + 3.0*mad, 0.08)
    return float(min(thr, 0.60))

def _normalized_deltas(areas_norm: np.ndarray) -> np.ndarray:
    if areas_norm.size < 2:
        return np.zeros(0, dtype=np.float64)
    return np.abs(np.diff(areas_norm))

def _segment_smooth_by_area(index_seq: List[int],
                            areas: Dict[int, int],
                            max_iters: Optional[int] = None,
                            verbose: bool = False) -> List[int]:
    """
    Smooth a contiguous sub-sequence (no empty-glottis frames inside) by relocating
    frames that cause outlier jumps in glottis area. Greedy local re-insertion
    minimizing adjacent area-mismatch cost.

    Keeps relative file access & renaming logic unchanged (we only permute indices).
    """
    n = len(index_seq)
    if n < 3:
        return index_seq[:]  # nothing to do

    maxA = float(max(areas[i] for i in index_seq)) or 1.0

    def norm_area(idx: int) -> float:
        return areas[idx] / maxA

    def seq_norm_areas(seq: List[int]) -> np.ndarray:
        return np.array([norm_area(i) for i in seq], dtype=np.float64)

    def insertion_cost(seq: List[int], j_idx: int, gap_pos: int) -> float:
        """
        Cost of inserting element j (global index) between seq[gap_pos] and seq[gap_pos+1].
        For boundaries, one-sided cost.
        """
        aj = norm_area(j_idx)
        if len(seq) == 0:
            return 0.0
        if gap_pos < 0:  # insert at beginning
            return abs(aj - norm_area(seq[0]))
        if gap_pos >= len(seq) - 1:  # insert at end
            return abs(norm_area(seq[-1]) - aj)
        al = norm_area(seq[gap_pos])
        ar = norm_area(seq[gap_pos+1])
        return abs(al - aj) + abs(aj - ar)

    def current_adjacent_cost(seq: List[int], pos: int) -> float:
        """Adjacent area cost around position pos (left and right, as available)."""
        a = norm_area(seq[pos])
        cost = 0.0
        if pos > 0:
            cost += abs(norm_area(seq[pos-1]) - a)
        if pos < len(seq) - 1:
            cost += abs(a - norm_area(seq[pos+1]))
        return cost

    seq = index_seq[:]
    iters = 0
    if max_iters is None:
        max_iters = 2 * n  # safe cap

    while iters < max_iters:
        iters += 1
        areas_norm = seq_norm_areas(seq)
        deltas = _normalized_deltas(areas_norm)
        thr = _robust_jump_threshold(deltas)
        # find worst jump
        if deltas.size == 0:
            break
        p = int(np.argmax(deltas))
        if deltas[p] <= thr:
            break  # sequence smooth enough

        # Consider moving the right frame (p+1) or the left frame (p)
        candidates = []
        for move_pos in (p, p+1):
            # compute current local cost that this element participates in
            cur_cost = current_adjacent_cost(seq, move_pos)

            # Evaluate best re-insertion gap across the sequence (excluding trivial no-op)
            best_gap = None
            best_cost = float("inf")
            # Build a temporary seq without the moving element
            j_idx = seq[move_pos]
            tmp = seq[:move_pos] + seq[move_pos+1:]
            for gap in range(-1, len(tmp)):  # -1 -> before first, len(tmp)-1 -> between last and end
                # avoid re-inserting back to the same local neighborhood (no-op)
                if move_pos-1 <= gap <= move_pos:
                    continue
                cost = insertion_cost(tmp, j_idx, gap)
                if cost < best_cost:
                    best_cost = cost
                    best_gap = gap

            # improvement if we actually reduce local mismatch by a margin
            improvement = cur_cost - best_cost
            candidates.append((improvement, move_pos, best_gap, best_cost))

        # choose the better move if it yields improvement
        candidates.sort(reverse=True, key=lambda x: x[0])
        best_impr, move_pos, best_gap, best_cost = candidates[0]
        if best_impr > 1e-6 and best_gap is not None:
            j_idx = seq.pop(move_pos)
            # re-insert at gap
            insert_pos = best_gap + 1  # turn "gap index" into list insertion index
            seq.insert(insert_pos, j_idx)
            if verbose:
                print(f"[smooth] moved frame@{move_pos} → insert@{insert_pos} (Δarea={deltas[p]:.3f} > thr {thr:.3f})")
            continue
        else:
            # Couldn't find beneficial move; stop to avoid churn
            break

    return seq

def _post_stabilize_sequence(final_idx_seq: List[int],
                             areas: Dict[int, int],
                             empty_set: set[int],
                             verbose: bool = False) -> List[int]:
    """
    Preserve the empty-glottis block as one chunk, and smooth the left/right
    non-empty segments independently.
    """
    if not empty_set:
        return _segment_smooth_by_area(final_idx_seq, areas, verbose=verbose)

    # Find contiguous empty block (by construction it should already be contiguous)
    pos_empty = [k for k, idx in enumerate(final_idx_seq) if idx in empty_set]
    if not pos_empty:
        return _segment_smooth_by_area(final_idx_seq, areas, verbose=verbose)

    e_start, e_end = min(pos_empty), max(pos_empty)
    left = final_idx_seq[:e_start]
    mid = final_idx_seq[e_start:e_end+1]   # empty block
    right = final_idx_seq[e_end+1:]

    left_s = _segment_smooth_by_area(left, areas, verbose=verbose)
    right_s = _segment_smooth_by_area(right, areas, verbose=verbose)

    if verbose:
        print(f"[smooth] empty block kept at [{e_start}:{e_end}] | left:{len(left)} right:{len(right)}")
    return left_s + mid + right_s

# =========================
# Core ordering in one folder (IN-PLACE RENAME)
# =========================

def order_bagls_greedy_dc_inplace(
    folder: str,
    lambda_area: float = 0.15,
    lambda_cent: float = 0.05,
    dilate_px: int = 1,
    verbose: bool = True,
    dry_run: bool = False,
) -> tuple[bool, int]:
    """
    IN-PLACE rename inside `folder` (e.g., ...\\organized_256 or ...\\croped_256).

    Steps:
      1) Collect pairs like 1328.png + 1328_seg.png
      2) Exclude empty-glottis masks, order non-empty by greedy nearest-neighbour using distance:
         D = (1 - Dice) + lambda_area * |Δarea|/maxA + lambda_cent * ||Δcentroid||/diag
      3) Insert empty-glottis frames at the 'closing' boundary (around minimal non-zero glottis area),
         or at start/end if that minimum is first/last.
      4) NEW: Post-stabilize the sequence by relocating frames that cause outlier jumps
         in normalized glottis area. Empty block remains intact.
      5) IN-PLACE two-phase rename to 1.png, 1_seg.png, 2.png, 2_seg.png, ...
         - Phase A: move all current files to unique temporary names (prevents collisions)
         - Phase B: move temps to final numbered names
      6) Write ordering_map.csv (temp->final + original names) for traceability.

    Returns:
      (ok, total_pairs)
    """
    if not os.path.isdir(folder):
        if verbose:
            print(f"[skip] Not a directory: {folder}")
        return False, 0

    # Gather pairs: any *.png that does NOT end with _seg.png and has a _seg.png partner
    rgb_list = sorted([p for p in glob(os.path.join(folder, "*.png"))
                       if not p.endswith("_seg.png") and not os.path.basename(p).startswith("._")])

    pairs: List[Tuple[str, str]] = []
    for rgb_path in rgb_list:
        base = os.path.splitext(os.path.basename(rgb_path))[0]
        seg_cand = os.path.join(folder, f"{base}_seg.png")
        if os.path.isfile(seg_cand):
            pairs.append((rgb_path, seg_cand))

    if len(pairs) == 0:
        if verbose:
            print(f"[skip] No pairs found in: {folder}")
        return False, 0

    # Precompute glottis masks + stats
    masks: Dict[int, np.ndarray] = {}
    areas: Dict[int, int] = {}
    cents: Dict[int, tuple[float, float]] = {}

    for idx, (_, seg_path) in enumerate(pairs):
        seg_img = _read_png_gray(seg_path)
        g = _extract_glottis_binary(seg_img)
        g = _maybe_dilate(g, dilate_px)
        masks[idx] = g
        areas[idx] = int(g.sum())
        cents[idx] = centroid(g)

    non_empty_idxs = [i for i in range(len(pairs)) if areas[i] > 0]
    empty_idxs = [i for i in range(len(pairs)) if areas[i] == 0]
    empty_set = set(empty_idxs)

    # If all empty → just sort by filename order and rename to 1..n
    if len(non_empty_idxs) == 0:
        index_sequence = list(range(len(pairs)))
        return _rename_in_place(folder, pairs, index_sequence, verbose=verbose, dry_run=dry_run)

    # Build distance for greedy NN over non-empty
    H, W = masks[non_empty_idxs[0]].shape
    diag = float(np.hypot(H, W))
    maxA_nonempty = float(max(areas[i] for i in non_empty_idxs))

    def _pair_distance(i: int, j: int) -> float:
        dc = dice_coef(masks[i], masks[j])
        area_term = abs(areas[i] - areas[j]) / (maxA_nonempty if maxA_nonempty > 0 else 1.0)
        ci, cj = cents[i], cents[j]
        if ci[0] < 0 or cj[0] < 0:
            cent_term = 1.0
        else:
            cent_term = np.hypot(ci[0]-cj[0], ci[1]-cj[1]) / (diag if diag > 0 else 1.0)
        return (1.0 - dc) + lambda_area * area_term + lambda_cent * cent_term

    current = non_empty_idxs[0]  # deterministic arbitrary start
    unused = set(non_empty_idxs[1:])
    ordered_non_empty = [current]
    while unused:
        nxt = min(unused, key=lambda j: _pair_distance(current, j))
        ordered_non_empty.append(nxt)
        unused.remove(nxt)
        current = nxt

    # Insert empties around the minimal non-zero area location
    min_area = min(areas[i] for i in ordered_non_empty)
    min_pos = next(k for k, i in enumerate(ordered_non_empty) if areas[i] == min_area)

    if empty_idxs:
        if min_pos == 0:
            final_idx_seq = empty_idxs + ordered_non_empty
        elif min_pos == len(ordered_non_empty) - 1:
            final_idx_seq = ordered_non_empty + empty_idxs
        else:
            left = ordered_non_empty[:min_pos+1]
            right = ordered_non_empty[min_pos+1:]
            final_idx_seq = left + empty_idxs + right
    else:
        final_idx_seq = ordered_non_empty

    # ---------- NEW: one quick stabilization pass to iron out big area jumps ----------
    final_idx_seq = _post_stabilize_sequence(final_idx_seq, areas, empty_set, verbose=verbose)
    # ----------------------------------------------------------------------------------

    return _rename_in_place(folder, pairs, final_idx_seq, verbose=verbose, dry_run=dry_run)

def _rename_in_place(
    folder: str,
    pairs: List[Tuple[str, str]],
    index_sequence: List[int],
    verbose: bool = True,
    dry_run: bool = False,
) -> tuple[bool, int]:
    """
    Two-phase in-place rename:
      A) rename all originals to unique temp names
      B) rename temps to final "1.png", "1_seg.png", ...
    Also writes ordering_map.csv with original -> temp -> final mappings.
    """
    # Guard: if already numbered 1..n exactly matching count, skip
    n = len(pairs)
    expected_rgb = {os.path.join(folder, f"{k}.png") for k in range(1, n+1)}
    expected_seg = {os.path.join(folder, f"{k}_seg.png") for k in range(1, n+1)}
    existing = set(glob(os.path.join(folder, "*.png")))
    if expected_rgb.issubset(existing) and expected_seg.issubset(existing):
        if verbose:
            print(f"[keep] Already appears ordered: {folder}")
        return True, n

    # Phase A: move originals to temp uniq names
    uid = uuid.uuid4().hex[:8]
    temp_map = []  # (orig_rgb, orig_seg, tmp_rgb, tmp_seg)
    for rgb_src, seg_src in pairs:
        tmp_rgb = os.path.join(folder, f"__tmp_{uid}__{os.path.basename(rgb_src)}")
        tmp_seg = os.path.join(folder, f"__tmp_{uid}__{os.path.basename(seg_src)}")
        temp_map.append((rgb_src, seg_src, tmp_rgb, tmp_seg))

    if verbose:
        print(f"[phase A] temp tagging {len(temp_map)} pairs in {folder}")

    if not dry_run:
        for orig_rgb, orig_seg, tmp_rgb, tmp_seg in temp_map:
            os.replace(orig_rgb, tmp_rgb)
            os.replace(orig_seg, tmp_seg)

    # Phase B: temp -> final numbered names following `index_sequence`
    if verbose:
        print(f"[phase B] assigning final numbers 1..{n} in {folder}")

    number_map = []  # rows for CSV
    for k, idx in enumerate(index_sequence, start=1):
        orig_rgb, orig_seg = pairs[idx]
        tmp_rgb = os.path.join(folder, f"__tmp_{uid}__{os.path.basename(orig_rgb)}")
        tmp_seg = os.path.join(folder, f"__tmp_{uid}__{os.path.basename(orig_seg)}")

        final_rgb = os.path.join(folder, f"{k}.png")
        final_seg = os.path.join(folder, f"{k}_seg.png")

        if not dry_run:
            os.replace(tmp_rgb, final_rgb)
            os.replace(tmp_seg, final_seg)

        number_map.append({
            "index": k,
            "orig_rgb": orig_rgb,
            "orig_seg": orig_seg,
            "temp_rgb": tmp_rgb,
            "temp_seg": tmp_seg,
            "final_rgb": final_rgb,
            "final_seg": final_seg,
        })

    # Safety: warn on leftovers (should be none)
    if not dry_run:
        leftovers = glob(os.path.join(folder, f"__tmp_{uid}__*.png"))
        for lf in leftovers:
            if verbose:
                print(f"[warn] leftover temp not reassigned: {lf}")

    # Write CSV map
    csv_path = os.path.join(folder, "ordering_map.csv")
    if not dry_run:
        with open(csv_path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(
                f,
                fieldnames=["index", "orig_rgb", "orig_seg", "temp_rgb", "temp_seg", "final_rgb", "final_seg"]
            )
            writer.writeheader()
            writer.writerows(number_map)

    if verbose:
        print(f"[done] {folder} → in-place rename complete ({n} pairs). Map: {csv_path}")
    return True, n

# =========================
# Batch over root with limit (unchanged)
# =========================

def order_all_bagls_sequences_inplace(
    root_organized_dir: str,
    inner_folder_names: tuple[str, ...] = ("organized_256", "croped_256"),
    limit: Optional[int] = None,
    lambda_area: float = 0.15,
    lambda_cent: float = 0.05,
    dilate_px: int = 1,
    verbose: bool = True,
    dry_run: bool = False,
) -> None:
    """
    Walks subfolders of `root_organized_dir`, finds the first existing inner folder
    among `inner_folder_names` (e.g., 'organized_256' or 'croped_256'), and performs
    IN-PLACE ordering/renaming inside it. Stops after `limit` processed subfolders.
    """
    subfolders = [p for p in glob(os.path.join(root_organized_dir, "*")) if os.path.isdir(p)]
    subfolders.sort()

    processed = 0
    for sf in subfolders:
        if limit is not None and processed >= limit:
            print(f"[stop] Reached limit={limit}")
            break

        candidate = None
        for name in inner_folder_names:
            test = os.path.join(sf, name)
            if os.path.isdir(test):
                candidate = test
                break

        if candidate is None:
            if verbose:
                print(f"[skip] No expected frames folder in: {sf} (looked for {inner_folder_names})")
            continue

        ok, n = order_bagls_greedy_dc_inplace(
            candidate,
            lambda_area=lambda_area,
            lambda_cent=lambda_cent,
            dilate_px=dilate_px,
            verbose=verbose,
            dry_run=dry_run,
        )
        if ok:
            processed += 1

    print(f"[summary] processed subfolders: {processed} (limit={limit})")

In [6]:
if __name__ == "__main__":
    # Example:
    order_all_bagls_sequences_inplace(
        r"D:\bagls-data\BAGLS\test_organized",
        inner_folder_names=("organized_256", "croped_256"),
        limit=None,                 # None = all
        lambda_area=0.15,
        lambda_cent=0.05,
        dilate_px=1,
        verbose=True,
        dry_run=False,            # True = show steps without renaming
    )
    pass

[smooth] moved frame@34 → insert@0 (Δarea=0.812 > thr 0.080)
[smooth] moved frame@35 → insert@0 (Δarea=0.822 > thr 0.080)
[smooth] moved frame@36 → insert@0 (Δarea=0.841 > thr 0.080)
[smooth] moved frame@37 → insert@0 (Δarea=0.837 > thr 0.080)
[smooth] moved frame@38 → insert@0 (Δarea=0.830 > thr 0.080)
[smooth] moved frame@39 → insert@44 (Δarea=0.860 > thr 0.080)
[smooth] moved frame@39 → insert@44 (Δarea=0.863 > thr 0.080)
[smooth] moved frame@39 → insert@42 (Δarea=0.859 > thr 0.080)
[smooth] moved frame@39 → insert@43 (Δarea=0.867 > thr 0.080)
[smooth] moved frame@39 → insert@8 (Δarea=0.882 > thr 0.080)
[smooth] moved frame@40 → insert@44 (Δarea=0.856 > thr 0.080)
[smooth] moved frame@40 → insert@44 (Δarea=0.859 > thr 0.080)
[smooth] moved frame@40 → insert@44 (Δarea=0.860 > thr 0.080)
[smooth] moved frame@40 → insert@44 (Δarea=0.867 > thr 0.080)
[smooth] moved frame@40 → insert@42 (Δarea=0.863 > thr 0.080)
[smooth] moved frame@40 → insert@42 (Δarea=0.856 > thr 0.080)
[smooth] moved